In [143]:
#Download Path
download_dir = r"C:\Users\Inv_edg\Downloads"  # Change this to your actual download directory
#Data Path
data_path = r"S:\Riew\Significance\Data.xlsx"
#VBA
excel_path = r"S:\Riew\Significance\AdjustingData - Copy.xlsm"
#ตลาดวันนี้เปิดแล้ว mkt = 1 ยังไม่เปิด mkt = 0
mkt = 1

import warnings
warnings.filterwarnings("ignore")

Download Recent Data

In [106]:
import os
import shutil
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.action_chains import ActionChains
# Set up Selenium webdriver
driver = webdriver.Chrome()  # You may need to change this path to the location of your Chrome webdriver

# Load the webpage
url = "https://www.set.or.th/en/market/statistics/short-sales/total-short-sales"
driver.get(url)
time.sleep(3)

# Find the export button
export_button = driver.find_element(By.XPATH, "//button[contains(text(),'Export Excel')]")

driver.execute_script("window.scrollBy(0, 500);")  # Scroll down by 500 pixels each time
time.sleep(1)
# Scroll to the export button
actions = ActionChains(driver)
actions.move_to_element(export_button).perform()

# Click the button
export_button.click()
time.sleep(3)

driver.close()
# Load the webpage
driver = webdriver.Chrome()
url = "https://www.set.or.th/en/market/statistics/short-sales/outstanding-short-positions"
driver.get(url)
time.sleep(3)

# Find the export button
export_button = driver.find_element(By.XPATH, "//button[contains(text(),'Export Excel')]")

driver.execute_script("window.scrollBy(0, 500);")  # Scroll down by 500 pixels each time
time.sleep(1)
# Scroll to the export button
actions = ActionChains(driver)
actions.move_to_element(export_button).perform()

# Click the button
export_button.click()
time.sleep(3)
driver.close()

In [107]:
name = ['SET:ADVANC', 'SET:AOT', 'SET:AWC', 'SET:BANPU', 'SET:BBL', 'SET:BDMS', 'SET:BEM', 'SET:BGRIM', 'SET:BH', 'SET:BTS', 'SET:CBG', 'SET:CENTEL', 'SET:COM7', 'SET:CPALL', 'SET:CPF', 'SET:CPN', 'SET:CRC', 'SET:DELTA', 'SET:EA', 'SET:EGCO', 'SET:GLOBAL', 'SET:GPSC', 'SET:GULF', 'SET:HMPRO', 'SET:INTUCH', 'SET:IVL', 'SET:KBANK', 'SET:KCE', 'SET:KTB', 'SET:KTC', 'SET:LH', 'SET:MINT', 'SET:MTC', 'SET:OR', 'SET:OSP', 'SET:PTT', 'SET:PTTEP', 'SET:PTTGC', 'SET:RATCH', 'SET:SAWAD', 'SET:SCB', 'SET:SCC', 'SET:SCGP', 'SET:TISCO', 'SET:TOP', 'SET:TRUE', 'SET:TTB', 'SET:TU', 'SET:WHA', 'SET:TLI']

VBA: Formatting File

In [108]:
import xlwings as xw

wb = xw.Book(excel_path)
macro_name = "AdjustExcelFilesRecent"

# Get the macro
macro = wb.macro(macro_name)

# Run the macro
macro()

In [109]:
import os

# Directory where the files are downloaded

# Get a list of all files in the download directory
files = os.listdir(download_dir)

# Initialize variables to store the paths of the most recent files
most_recent_outstanding_short_positions = None
most_recent_short_sales = None

# Filter files starting with "Outstanding ShortPositions" or "Short Sales"
for file in files:
    if file.startswith("Outstanding Short Positions"):
        file_path = os.path.join(download_dir, file)
        if most_recent_outstanding_short_positions is None or os.path.getmtime(file_path) > os.path.getmtime(most_recent_outstanding_short_positions):
            most_recent_outstanding_short_positions = file_path
    elif file.startswith("Short Sales"):
        file_path = os.path.join(download_dir, file)
        if most_recent_short_sales is None or os.path.getmtime(file_path) > os.path.getmtime(most_recent_short_sales):
            most_recent_short_sales = file_path

# Now you have the paths to the most recent files for each type
print("Most recent Outstanding ShortPositions file:", most_recent_outstanding_short_positions)
print("Most recent Short Sales file:", most_recent_short_sales)

Most recent Outstanding ShortPositions file: C:\Users\Inv_edg\Downloads\Outstanding Short Positions_17-06-2024.xlsx
Most recent Short Sales file: C:\Users\Inv_edg\Downloads\Short Sales_17-06-2024 to 17-06-2024.xlsx


In [159]:
import pandas as pd
allX_noshift = pd.read_excel(data_path)

Concat and save updated data

In [161]:
import openpyxl
import math
import numpy as np
import os

df_temp = pd.DataFrame()

filename = os.path.basename(most_recent_short_sales)
date_part = filename.split('_')[1].split(' to ')[0]
df_temp['dateexcel'] = [date_part]


for j in range(len(name)):
    volume = []
    turnover = []
    volume_compare_automatching = []
    outstanding_short_volume = []
    paid_up_shares = []
    Outstanding_short_volume_to_paid_up_share = []
    outstanding_short_local_and_NVDR_Volume_to_paid_up_share = []

    stock = name[j]
    stock = stock[4:]

    path = most_recent_short_sales
    wb = openpyxl.load_workbook(path)
        # Select the active worksheet
    ws = wb.active

        # Iterate through column A to find the cell with the value same as stock
    stock_found = False

    for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=1):
        for cell in row:
            if cell.value == stock:
                # Get values from columns B, C, D on the same row and append to lists
                volume.append(ws.cell(row=cell.row, column=2).value)
                turnover.append(ws.cell(row=cell.row, column=3).value)
                volume_compare_automatching.append(ws.cell(row=cell.row, column=4).value)
                stock_found = True  # Set flag to True if stock is found
                break  # Stop searching once the value is found
        if stock_found:
            break  # Exit the outer loop if stock is found
        # Check if stock is not found in the entire column
    if not stock_found:
        # Append NaN if stock is not found
        volume.append(math.nan)
        turnover.append(math.nan)
        volume_compare_automatching.append(math.nan)

        # Reset the flag for the next iteration
    wb.close()

    path = most_recent_outstanding_short_positions
    wb = openpyxl.load_workbook(path)
    # Select the active worksheet
    ws = wb.active
    # Iterate through column A to find the cell with the value same as stock
    stock_found = False

    for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=1):
        for cell in row:
            if cell.value == stock:
                # Get values from columns B, C, D, and E on the same row and append to lists
                outstanding_short_volume.append(ws.cell(row=cell.row, column=2).value)
                paid_up_shares.append(ws.cell(row=cell.row, column=3).value)
                Outstanding_short_volume_to_paid_up_share.append(ws.cell(row=cell.row, column=4).value)
                outstanding_short_local_and_NVDR_Volume_to_paid_up_share.append(ws.cell(row=cell.row, column=5).value)
                stock_found = True  # Set flag to True if stock is found
                break  # Stop searching once the value is found
        if stock_found:
            break 
        # Check if stock is not found in the entire column
    if not stock_found:
        # Append NaN if stock is not found
        outstanding_short_volume.append(math.nan)
        paid_up_shares.append(math.nan)
        Outstanding_short_volume_to_paid_up_share.append(math.nan)
        outstanding_short_local_and_NVDR_Volume_to_paid_up_share.append(math.nan)

        # Reset the flag for the next iteration
    wb.close()

    df_temp[name[j]+'Short_vol'] = np.array(volume)
    df_temp[name[j]+'Turnover'] = np.array(turnover)
    df_temp[name[j]+'Vol to Automatching'] = np.array(volume_compare_automatching)
    df_temp[name[j]+'Outstanding Short_vol'] = np.array(outstanding_short_volume)
    df_temp[name[j]+'Paid Up Shares'] = np.array(paid_up_shares)
    df_temp[name[j]+'Outstanding Short_vol to Paid Up Shares'] = np.array(Outstanding_short_volume_to_paid_up_share)
    df_temp[name[j]+'outstanding_short_local_and_NVDR_Volume to Paid Up Shares'] = np.array(outstanding_short_local_and_NVDR_Volume_to_paid_up_share)

if allX_noshift['dateexcel'].iloc[-1] == df_temp['dateexcel'].iloc[0]:
    pass
else:
    allX_noshift = pd.concat([allX_noshift, df_temp])


In [113]:
allX_noshift.to_excel(r"S:\Riew\Significance\Data.xlsx", index=False)

In [162]:
allX_noshift

,dateexcel,SET:ADVANCShort_vol,SET:ADVANCTurnover,SET:ADVANCVol to Automatching,SET:ADVANCOutstanding Short_vol,SET:ADVANCPaid Up Shares,SET:ADVANCOutstanding Short_vol to Paid Up Shares,SET:ADVANCoutstanding_short_local_and_NVDR_Volume to Paid Up Shares,SET:AOTShort_vol,SET:AOTTurnover,...,SET:WHAPaid Up Shares,SET:WHAOutstanding Short_vol to Paid Up Shares,SET:WHAoutstanding_short_local_and_NVDR_Volume to Paid Up Shares,SET:TLIShort_vol,SET:TLITurnover,SET:TLIVol to Automatching,SET:TLIOutstanding Short_vol,SET:TLIPaid Up Shares,SET:TLIOutstanding Short_vol to Paid Up Shares,SET:TLIoutstanding_short_local_and_NVDR_Volume to Paid Up Shares
0,02-04-2024,452600,92505400,0.142255,1493900,2974209736,0.05,0.44,797900,52283100,...,14946834679,0.23,0.64,755900,6816140,0.088595,1734300,11450000000,0.02,0.07
1,03-04-2024,563900,114893300,0.077123,1503400,2974209736,0.05,0.44,1818600,119124800,...,14946834679,0.23,0.65,798400,7242415,0.036914,1485800,11450000000,0.01,0.07
2,04-04-2024,392100,80124000,0.104842,1702900,2974209736,0.06,0.45,2449200,159244650,...,14946834679,0.23,0.67,947700,8451950,0.073113,1400000,11450000000,0.01,0.07
3,05-04-2024,44300,9022800,0.017693,1516400,2974209736,0.05,0.44,687300,45037225,...,14946834679,0.23,0.67,195700,1741730,0.037500,1000000,11450000000,0.01,0.07
4,09-04-2024,778100,159607500,0.093814,1764200,2974209736,0.06,0.46,998900,66293850,...,14946834679,0.22,0.67,550700,4941195,0.030164,1051400,11450000000,0.01,0.07
5,10-04-2024,460500,94844800,0.049047,1701700,2974209736,0.06,0.47,2326200,156075500,...,14946834679,0.26,0.73,897400,8198320,0.105337,1101700,11450000000,0.01,0.07
6,11-04-2024,522900,106375400,0.111182,1821100,2974209736,0.06,0.50,1917400,128191550,...,14946834679,0.26,0.72,1401100,12793015,0.117049,1141200,11450000000,0.01,0.07
7,17-04-2024,673300,135344000,0.078260,1744800,2974209736,0.06,0.51,3437200,225497625,...,14946834679,0.26,0.71,1381100,12468375,0.077895,1000000,11450000000,0.01,0.07
8,18-04-2024,862100,170112200,0.098386,1869600,2974209736,0.06,0.52,3558100,233172525,...,14946834679,0.27,0.76,2423300,21651495,0.120873,2219100,11450000000,0.02,0.09
9,19-04-2024,567400,110893450,0.071546,1747000,2974209736,0.06,0.53,3961900,252923750,...,14946834679,0.28,0.83,2531600,22225595,0.094117,3136500,11450000000,0.03,0.10


Shift Data for backtest purpose

In [163]:
allX = allX_noshift.copy()
for j in range(len(name)):
    allX[name[j]+'Short_vol'] = allX[name[j]+'Short_vol'].shift(1)
    allX[name[j]+'Turnover'] = allX[name[j]+'Turnover'].shift(1)
    allX[name[j]+'Vol to Automatching'] = allX[name[j]+'Vol to Automatching'].shift(1)
    allX[name[j]+'Outstanding Short_vol'] = allX[name[j]+'Outstanding Short_vol'].shift(1)
    allX[name[j]+'Paid Up Shares'] = allX[name[j]+'Paid Up Shares'].shift(1)
    allX[name[j]+'Outstanding Short_vol to Paid Up Shares'] = allX[name[j]+'Outstanding Short_vol to Paid Up Shares'].shift(1)
    allX[name[j]+'outstanding_short_local_and_NVDR_Volume to Paid Up Shares'] = allX[name[j]+'outstanding_short_local_and_NVDR_Volume to Paid Up Shares'].shift(1)
newdata = allX_noshift.iloc[-2:]
newdata

,dateexcel,SET:ADVANCShort_vol,SET:ADVANCTurnover,SET:ADVANCVol to Automatching,SET:ADVANCOutstanding Short_vol,SET:ADVANCPaid Up Shares,SET:ADVANCOutstanding Short_vol to Paid Up Shares,SET:ADVANCoutstanding_short_local_and_NVDR_Volume to Paid Up Shares,SET:AOTShort_vol,SET:AOTTurnover,...,SET:WHAPaid Up Shares,SET:WHAOutstanding Short_vol to Paid Up Shares,SET:WHAoutstanding_short_local_and_NVDR_Volume to Paid Up Shares,SET:TLIShort_vol,SET:TLITurnover,SET:TLIVol to Automatching,SET:TLIOutstanding Short_vol,SET:TLIPaid Up Shares,SET:TLIOutstanding Short_vol to Paid Up Shares,SET:TLIoutstanding_short_local_and_NVDR_Volume to Paid Up Shares
45,14-06-2024,100,21000,0.000055,1113300,2974209736,0.04,0.22,1258300,75855375,...,14946834679,0.30,0.64,1510700,11602000,0.107758,9785900,11450000000,0.09,0.20
46,17-06-2024,774600,162973400,0.114376,1187900,2974209736,0.04,0.21,2567100,153281600,...,14946834679,0.31,0.69,771100,5830200,0.050533,9286100,11450000000,0.08,0.19


In [119]:
from tvDatafeed import TvDatafeed, Interval
from math import sqrt

tv = TvDatafeed()
df = tv.get_hist(symbol='SET:SET50', exchange='SET', interval=Interval.in_daily, n_bars=len(allX) + 1)
set50 = df[['close']]
set50.rename(columns = {'close':'set50'}, inplace=True)
#load all set50 data
for i in range (len(name)):
    try:
        df = tv.get_hist(symbol=name[i], exchange='SET', interval=Interval.in_daily, n_bars=len(allX) + 1)
        df = df[['open','close']]
        df[name[i]+'_return'] = (df['close'] - df['open']) / df['open'] * 100
        df[name[i]+'_lag1'] = df[name[i]+'_return'].shift(1)
        set50 = pd.merge(set50, df, left_index=True, right_index=True, how='outer')
        set50.rename(columns = {'open':name[i]+'_open'}, inplace=True)
        set50.rename(columns = {'close':name[i]+'_close'}, inplace=True)
        print(i)
    except:
        df = tv.get_hist(symbol=name[i], exchange='SET', interval=Interval.in_daily, n_bars=len(allX) + 1)
        df = df[['open','close']]
        df[name[i]+'_return'] = (df['close'] - df['open']) / df['open'] * 100
        df[name[i]+'_lag1'] = df[name[i]+'_return'].shift(1)
        set50 = pd.merge(set50, df, left_index=True, right_index=True, how='outer')
        set50.rename(columns = {'open':name[i]+'_open'}, inplace=True)
        set50.rename(columns = {'close':name[i]+'_close'}, inplace=True)
        print(i)
set50 = set50.drop('set50',axis=1)
#returns = np.log(set50).diff() * 100 #1st difference

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [120]:
set50 = set50.reset_index(drop=False)
set50['datetime'] = set50['datetime'].dt.strftime('%d-%m-%Y')
set50 = set50.rename(columns={'datetime':'dateexcel'})
# Display the updated dataframe
set50.tail(5)

,dateexcel,SET:ADVANC_open,SET:ADVANC_close,SET:ADVANC_return,SET:ADVANC_lag1,SET:AOT_open,SET:AOT_close,SET:AOT_return,SET:AOT_lag1,SET:AWC_open,...,SET:TU_return,SET:TU_lag1,SET:WHA_open,SET:WHA_close,SET:WHA_return,SET:WHA_lag1,SET:TLI_open,SET:TLI_close,SET:TLI_return,SET:TLI_lag1
43,12-06-2024,211.0,210.0,-0.473934,0.476190,60.00,59.50,-0.833333,-0.413223,3.54,...,1.307190,-1.923077,5.30,5.30,0.000000,-1.851852,7.80,7.50,-3.846154,-2.484472
44,13-06-2024,210.0,209.0,-0.476190,-0.473934,59.75,60.00,0.418410,-0.833333,3.54,...,1.298701,1.307190,5.30,5.25,-0.943396,0.000000,7.55,7.60,0.662252,-3.846154
45,14-06-2024,210.0,211.0,0.476190,-0.476190,60.00,60.00,0.000000,0.418410,3.48,...,-0.636943,1.298701,5.25,5.30,0.952381,-0.943396,7.65,7.60,-0.653595,0.662252
46,17-06-2024,212.0,209.0,-1.415094,0.476190,60.00,60.50,0.833333,0.000000,3.42,...,-1.290323,-0.636943,5.30,5.30,0.000000,0.952381,7.60,7.55,-0.657895,-0.653595
47,18-06-2024,209.0,209.0,0.000000,-1.415094,60.75,61.25,0.823045,0.833333,3.46,...,1.298701,-1.290323,5.30,5.30,0.000000,0.000000,7.55,7.60,0.662252,-0.657895


In [121]:
allX = allX.reset_index(drop = True)
allX.tail()

,dateexcel,SET:ADVANCShort_vol,SET:ADVANCTurnover,SET:ADVANCVol to Automatching,SET:ADVANCOutstanding Short_vol,SET:ADVANCPaid Up Shares,SET:ADVANCOutstanding Short_vol to Paid Up Shares,SET:ADVANCoutstanding_short_local_and_NVDR_Volume to Paid Up Shares,SET:AOTShort_vol,SET:AOTTurnover,...,SET:WHAPaid Up Shares,SET:WHAOutstanding Short_vol to Paid Up Shares,SET:WHAoutstanding_short_local_and_NVDR_Volume to Paid Up Shares,SET:TLIShort_vol,SET:TLITurnover,SET:TLIVol to Automatching,SET:TLIOutstanding Short_vol,SET:TLIPaid Up Shares,SET:TLIOutstanding Short_vol to Paid Up Shares,SET:TLIoutstanding_short_local_and_NVDR_Volume to Paid Up Shares
42,11-06-2024,756900.0,159281100.0,0.100962,614000.0,2.974210e+09,0.02,0.21,2385900.0,145377000.0,...,1.494683e+10,0.30,0.66,1688700.0,13694670.0,0.090580,6289200.0,1.145000e+10,0.05,0.12
43,12-06-2024,1396500.0,294482200.0,0.162812,1341900.0,2.974210e+09,0.05,0.24,1926800.0,116559525.0,...,1.494683e+10,0.30,0.68,1049200.0,8330120.0,0.028109,5100800.0,1.145000e+10,0.04,0.12
44,13-06-2024,545000.0,114779700.0,0.109403,1128600.0,2.974210e+09,0.04,0.23,4185700.0,249835675.0,...,1.494683e+10,0.27,0.65,7604400.0,57799705.0,0.153160,11401000.0,1.145000e+10,0.10,0.20
45,14-06-2024,497700.0,104511900.0,0.194209,1221700.0,2.974210e+09,0.04,0.22,2005300.0,119913850.0,...,1.494683e+10,0.31,0.63,1178100.0,8875120.0,0.058038,9270300.0,1.145000e+10,0.08,0.19
46,17-06-2024,100.0,21000.0,0.000055,1113300.0,2.974210e+09,0.04,0.22,1258300.0,75855375.0,...,1.494683e+10,0.30,0.64,1510700.0,11602000.0,0.107758,9785900.0,1.145000e+10,0.09,0.20


X and Y

In [179]:
df3 = pd.DataFrame()

for i in range(len(name)):
    x = allX[['dateexcel',name[i]+'Short_vol',name[i]+'Outstanding Short_vol',name[i]+'Outstanding Short_vol to Paid Up Shares',name[i]+'Vol to Automatching', name[i]+'outstanding_short_local_and_NVDR_Volume to Paid Up Shares']].dropna()
    y = set50[['dateexcel',name[i]+'_open',name[i]+'_close',name[i]+'_return',name[i]+'_lag1']]
    y = y[:-mkt] #มีถ้าตลาดเปิดไปแล้ว
    x = x.apply(lambda col: pd.to_numeric(col, errors='coerce') if col.name != 'dateexcel' else col)
    x[name[i]+'Short_vol'] = x[name[i]+'Short_vol'].pct_change() * 100
    x[name[i]+'Outstanding Short_vol'] = x[name[i]+'Outstanding Short_vol'].pct_change() * 100
    x[name[i]+'Outstanding Short_vol to Paid Up Shares'] = x[name[i]+'Outstanding Short_vol to Paid Up Shares'].pct_change() * 100
    x[name[i]+'Vol to Automatching'] = x[name[i]+'Vol to Automatching'].pct_change() * 100
    x[name[i]+'outstanding_short_local_and_NVDR_Volume to Paid Up Shares'] = x[name[i]+'outstanding_short_local_and_NVDR_Volume to Paid Up Shares'].pct_change() * 100


    merged_df = pd.merge(y, x, on='dateexcel', how='inner')
    merged_df = merged_df.replace([np.inf, -np.inf], np.nan).dropna()
    merged_df.reset_index(drop=True, inplace=True)

    merged_df.rename(columns={name[i]+'_open': 'Open'}, inplace=True)
    merged_df.rename(columns={name[i]+'_close': 'Close'}, inplace=True)
    merged_df.rename(columns={name[i]+'_return': 'Return'}, inplace=True)
    merged_df.rename(columns={name[i]+'_lag1': 'Lag1'}, inplace=True)

    merged_df.rename(columns={name[i]+'Short_vol': 'Short_vol'}, inplace=True)
    merged_df.rename(columns={name[i]+'Outstanding Short_vol': 'Outstanding Short_vol'}, inplace=True)
    merged_df.rename(columns={name[i]+'Outstanding Short_vol to Paid Up Shares': 'Outstanding Short_vol to Paid Up Shares'}, inplace=True)
    merged_df.rename(columns={name[i]+'Vol to Automatching': 'Vol to Automatching'}, inplace=True)
    merged_df.rename(columns={name[i]+'outstanding_short_local_and_NVDR_Volume to Paid Up Shares': 'outstanding_short_local_and_NVDR_Volume to Paid Up Shares'}, inplace=True)
    merged_df['Stock'] = name[i]
    #merged_df['']
    if i == 0:
        df3 = merged_df.copy()
    else:
        df3 = pd.concat([df3, merged_df], ignore_index=True)


x = df3[['dateexcel','Stock','Short_vol','Outstanding Short_vol','Outstanding Short_vol to Paid Up Shares','Vol to Automatching', 'outstanding_short_local_and_NVDR_Volume to Paid Up Shares']].dropna()
x.iloc[:, 2:] = x.iloc[:, 2:].apply(pd.to_numeric)
y = df3[['dateexcel','Stock','Open','Close','Return','Lag1']]

Strategy Testing

In [193]:
avg_gain = 0
avg_loss = 0
avg_all = 0
avg_gain_count = 0
avg_loss_count = 0
avg_all_count = 0
winlist = []
drawlist=[]
loselist=[]
profit_list = []
profitTotal = []
tradecount = 0
shortcount = 0
longwinlosecount = 0
shortwinlosecount = 0
longwincount = 0
shortwincount = 0
notional = 100000
df_summary = pd.DataFrame()
for i in range(len(name)):
    df_tempx = x[x['Stock'] == name[i]].reset_index(drop=True)
    df_tempy = y[y['Stock'] == name[i]].reset_index(drop=True)
            #check win rate of 1 side by remove other side order
    adayprofit = []
    for j in range(len(df_tempx)):#df_tempy['Lag1'][j] df_tempx['outstanding_short_local_and_NVDR_Volume to Paid Up Shares]
        heehe = df_tempy['Close'][j]-df_tempy['Open'][j] #df_tempx['Outstanding Short_vol']
        if df_tempx['Outstanding Short_vol'][j] > 5 and df_tempx['outstanding_short_local_and_NVDR_Volume to Paid Up Shares'][j] >= -8: 
            heehe = -heehe
            shortcount += 1
            if heehe != 0:
                shortwinlosecount += 1
            if heehe > 0:
                shortwincount +=1
        elif df_tempx['outstanding_short_local_and_NVDR_Volume to Paid Up Shares'][j] < -8 and df_tempx['Outstanding Short_vol'][j] <= 5:
            heehe = heehe
            tradecount += 1
            if heehe != 0:
                longwinlosecount += 1
            if heehe >0:
                longwincount += 1
        else:
            heehe = 0
        adayprofit.append(heehe)
    df_tempy['profit'] = np.array(adayprofit)

    df_temp = pd.merge(df_tempx, df_tempy, on='dateexcel')

    asdasd = []
    for j in range(len(df_temp)):
        num = notional / df_temp['Open'][j]
        head = df_temp['profit'][j] * num
        asdasd.append(round(head,2))

    win_num = 0
    draw_num = 0
    lose_num = 0
    for j in range(len(df_temp)):
        if df_temp['profit'][j] > 0:
            win_num += 1
        elif df_temp['profit'][j] <0:
            lose_num += 1
        else:
            draw_num += 1
    winlist.append(win_num)
    drawlist.append(draw_num)
    loselist.append(lose_num)
    df_temp['profit100,000'] = np.array(asdasd)
    profit = np.sum(df_temp['profit100,000'])
    profit_list.append(round(profit, 2))
    df_summary = pd.concat([df_summary, df_temp])
df_summary = df_summary.reset_index()

for cat in range (len(df_summary)):
    if df_summary['profit100,000'][cat] > 0:
        avg_gain_count += 1
        avg_gain = avg_gain + df_summary['profit100,000'][cat]
    elif df_summary['profit100,000'][cat] < 0:
        avg_loss_count += 1
        avg_loss = avg_loss + df_summary['profit100,000'][cat]

df_profit = pd.DataFrame()
df_profit['Stock'] = np.array(name)
df_profit['Profit(Notional 100,000)'] = np.array(profit_list)
df_profit['Win'] = np.array(winlist)
df_profit['Draw'] = np.array(drawlist)
df_profit['Loss'] = np.array(loselist)
df_profit['win/win+loss'] = (df_profit['Win'] / (df_profit['Win'] + df_profit['Loss']) * 100).round(2)
df_profit['Gain/Loss'] = df_profit['Profit(Notional 100,000)'].apply(lambda x: 1 if x > 0 else 0)
Total = np.sum(df_profit['Profit(Notional 100,000)'])
Total_count = np.sum(df_profit['Gain/Loss'])
Total_win = (np.sum(df_profit['Win']) / (np.sum(df_profit['Win']) + np.sum(df_profit['Loss'])) * 100).round(2)

average_gain = avg_gain / avg_gain_count
average_loss = avg_loss / avg_loss_count
average_all = Total / (tradecount + shortcount)

winratelong = round(longwincount/longwinlosecount * 100,2)
winrateshort = round(shortwincount/shortwinlosecount * 100,2)
results_dict = {
    'Total': [Total],
    'Total number of profit stocks': [str(Total_count) + '/50'],
    'Total winrate (%)': [Total_win],
    'n. Long' : [tradecount],
    'n. Short': [shortcount],
    'Average_gain' : [average_gain],
    'Average_loss' : [average_loss],
    'Average' : [average_all],
    'Win Rate (Long)' : [winratelong],
    'Win Rate (Short)' : [winrateshort],
}

# Convert the dictionary to a DataFrame
results_df = pd.DataFrame(results_dict)
results_df

,Total,Total number of profit stocks,Total winrate (%),n. Long,n. Short,Average_gain,Average_loss,Average,Win Rate (Long),Win Rate (Short)
0,184523.05,38/50,59.23,87,616,1388.971471,-1229.603632,262.479445,54.29,59.92


In [192]:
#Loop parameter
woww = 0
for riew in range(0, 10):

    for john in range(0, 10):

        winlist = []
        drawlist=[]
        loselist=[]
        profit_list = []
        profitTotal = []
        tradecount = 0
        shortcount = 0
        notional = 100000
        df_summary = pd.DataFrame()
        for i in range(len(name)):
            df_tempx = x[x['Stock'] == name[i]].reset_index(drop=True)
            df_tempy = y[y['Stock'] == name[i]].reset_index(drop=True)
            #check win rate of 1 side by remove other side order
            adayprofit = []
            for j in range(len(df_tempx)):#df_tempy['Lag1'][j] df_tempx['outstanding_short_local_and_NVDR_Volume to Paid Up Shares]
                heehe = df_tempy['Close'][j]-df_tempy['Open'][j] #df_tempx['Outstanding Short_vol']
                if df_tempx['outstanding_short_local_and_NVDR_Volume to Paid Up Shares'][j] >= riew: 
                    heehe = -heehe
                    shortcount += 1
                elif df_tempx['outstanding_short_local_and_NVDR_Volume to Paid Up Shares'][j] <= john:
                    heehe = heehe
                    tradecount += 1
                else:
                    heehe = 0
                adayprofit.append(heehe)
            df_tempy['profit'] = np.array(adayprofit)

            df_temp = pd.merge(df_tempx, df_tempy, on='dateexcel')

            asdasd = []
            for j in range(len(df_temp)):
                num = notional / df_temp['Open'][j]
                head = df_temp['profit'][j] * num
                asdasd.append(round(head,2))

            win_num = 0
            draw_num = 0
            lose_num = 0
            for j in range(len(df_temp)):
                if df_temp['profit'][j] > 0:
                    win_num += 1
                elif df_temp['profit'][j] <0:
                    lose_num += 1
                else:
                    draw_num += 1
            winlist.append(win_num)
            drawlist.append(draw_num)
            loselist.append(lose_num)
            df_temp['profit100,000'] = np.array(asdasd)
            profit = np.sum(df_temp['profit100,000'])
            profit_list.append(round(profit, 2))
            df_summary = pd.concat([df_summary, df_temp])
        df_temp

        df_profit = pd.DataFrame()
        df_profit['Stock'] = np.array(name)
        df_profit['Profit(Notional 100,000)'] = np.array(profit_list)
        df_profit['Win'] = np.array(winlist)
        df_profit['Draw'] = np.array(drawlist)
        df_profit['Loss'] = np.array(loselist)
        df_profit['win/win+loss'] = (df_profit['Win'] / (df_profit['Win'] + df_profit['Loss']) * 100).round(2)
        df_profit['Gain/Loss'] = df_profit['Profit(Notional 100,000)'].apply(lambda x: 1 if x > 0 else 0)
        Total = np.sum(df_profit['Profit(Notional 100,000)'])
        Total_count = np.sum(df_profit['Gain/Loss'])
        Total_win = (np.sum(df_profit['Win']) / (np.sum(df_profit['Win']) + np.sum(df_profit['Loss'])) * 100).round(2)

        results_dict = {
            'Total': [Total],
            'Total number of profit stocks': [str(Total_count) + '/50'],
            'Total winrate (%)': [Total_win]
        }

        # Convert the dictionary to a DataFrame
        results_df = pd.DataFrame(results_dict)
        print(riew, john, tradecount, shortcount)
        print(results_df)
        print("")
        if results_df['Total winrate (%)'][0] > woww:
            df_yes = results_df.copy
            woww = results_df['Total winrate (%)'][0]
            para1 = riew
            para2 = john
print(woww, para1, para2)

0 0 753 1458
      Total Total number of profit stocks  Total winrate (%)
0  270353.1                         38/50              53.56

0 1 753 1458
      Total Total number of profit stocks  Total winrate (%)
0  270353.1                         38/50              53.56

0 2 753 1458
      Total Total number of profit stocks  Total winrate (%)
0  270353.1                         38/50              53.56

0 3 753 1458
      Total Total number of profit stocks  Total winrate (%)
0  270353.1                         38/50              53.56

0 4 753 1458
      Total Total number of profit stocks  Total winrate (%)
0  270353.1                         38/50              53.56

0 5 753 1458
      Total Total number of profit stocks  Total winrate (%)
0  270353.1                         38/50              53.56

0 6 753 1458
      Total Total number of profit stocks  Total winrate (%)
0  270353.1                         38/50              53.56

0 7 753 1458
      Total Total number of profit 

In [185]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

xs = x[['Short_vol','Outstanding Short_vol','Outstanding Short_vol to Paid Up Shares','Vol to Automatching', 'outstanding_short_local_and_NVDR_Volume to Paid Up Shares']]
ys = y[['Return']]

#xs = x[['outstanding_short_local_and_NVDR_Volume to Paid Up Shares']]
xs = sm.add_constant(xs)

# Fit the OLS model
model = sm.OLS(ys, xs)

# Perform the fitting
results = model.fit()

# Print a summary of the regression results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 Return   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.995
Date:                Tue, 18 Jun 2024   Prob (F-statistic):             0.0764
Time:                        11:08:11   Log-Likelihood:                -4015.0
No. Observations:                2211   AIC:                             8042.
Df Residuals:                    2205   BIC:                             8076.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                                                coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------

In [182]:
x['dateexcel'] = pd.to_datetime(x['dateexcel'], format='%d-%m-%Y')
y['dateexcel'] = pd.to_datetime(y['dateexcel'], format='%d-%m-%Y')
cutoff_date = pd.to_datetime('2024-06-06')
x_test = x[x['dateexcel'] > cutoff_date]
y_test = y[y['dateexcel'] > cutoff_date]
x_train = x[x['dateexcel'] <= cutoff_date]
y_train = y[y['dateexcel'] <= cutoff_date]

In [187]:
xs = x_train[['outstanding_short_local_and_NVDR_Volume to Paid Up Shares']]
ys = y_train[['Return']]

#xs = x[['outstanding_short_local_and_NVDR_Volume to Paid Up Shares']]
xs = sm.add_constant(xs)

# Fit the OLS model
model = sm.OLS(ys, xs)

# Perform the fitting
results = model.fit()

# Print a summary of the regression results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 Return   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     14.36
Date:                Tue, 18 Jun 2024   Prob (F-statistic):           0.000156
Time:                        11:09:03   Log-Likelihood:                -3359.3
No. Observations:                1862   AIC:                             6723.
Df Residuals:                    1860   BIC:                             6734.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                                                                coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------

Today's action

In [164]:
columns_to_transform = newdata.columns[newdata.columns != 'dateexcel']
newdata[columns_to_transform] = newdata[columns_to_transform].pct_change() * 100
newdata = newdata.tail(1)
newdata

,dateexcel,SET:ADVANCShort_vol,SET:ADVANCTurnover,SET:ADVANCVol to Automatching,SET:ADVANCOutstanding Short_vol,SET:ADVANCPaid Up Shares,SET:ADVANCOutstanding Short_vol to Paid Up Shares,SET:ADVANCoutstanding_short_local_and_NVDR_Volume to Paid Up Shares,SET:AOTShort_vol,SET:AOTTurnover,...,SET:WHAPaid Up Shares,SET:WHAOutstanding Short_vol to Paid Up Shares,SET:WHAoutstanding_short_local_and_NVDR_Volume to Paid Up Shares,SET:TLIShort_vol,SET:TLITurnover,SET:TLIVol to Automatching,SET:TLIOutstanding Short_vol,SET:TLIPaid Up Shares,SET:TLIOutstanding Short_vol to Paid Up Shares,SET:TLIoutstanding_short_local_and_NVDR_Volume to Paid Up Shares
45,14-06-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,17-06-2024,774500.0,775963.809524,207595.372394,6.700799,0.0,0.0,-4.545455,104.013351,102.070849,...,0.0,3.333333,7.8125,-48.957437,-49.748319,-53.104612,-5.107348,0.0,-11.111111,-5.0


In [221]:
df4 = pd.DataFrame()
for i in range(len(name)):
    ex = newdata[['dateexcel',name[i]+'Short_vol',name[i]+'Outstanding Short_vol',name[i]+'Outstanding Short_vol to Paid Up Shares',name[i]+'Vol to Automatching', name[i]+'outstanding_short_local_and_NVDR_Volume to Paid Up Shares']].dropna()
    ex = ex.apply(lambda col: pd.to_numeric(col, errors='coerce') if col.name != 'dateexcel' else col)
    merged_df = ex.copy()
    merged_df = merged_df.replace([np.inf, -np.inf], np.nan).dropna()
    merged_df.reset_index(drop=True, inplace=True)

    merged_df.rename(columns={name[i]+'Short_vol': 'Short_vol'}, inplace=True)
    merged_df.rename(columns={name[i]+'Outstanding Short_vol': 'Outstanding Short_vol'}, inplace=True)
    merged_df.rename(columns={name[i]+'Outstanding Short_vol to Paid Up Shares': 'Outstanding Short_vol to Paid Up Shares'}, inplace=True)
    merged_df.rename(columns={name[i]+'Vol to Automatching': 'Vol to Automatching'}, inplace=True)
    merged_df.rename(columns={name[i]+'outstanding_short_local_and_NVDR_Volume to Paid Up Shares': 'outstanding_short_local_and_NVDR_Volume to Paid Up Shares'}, inplace=True)
    merged_df['Stock'] = name[i]
    if i == 0:
        df4 = merged_df.copy()
    else:
        df4 = pd.concat([df4, merged_df], ignore_index=True)
df4['Main Strategy'] = np.where((df4['Outstanding Short_vol'] <= 5) & (df4['outstanding_short_local_and_NVDR_Volume to Paid Up Shares'] < -8), 'Long',
                            np.where((df4['Outstanding Short_vol'] > 5) & (df4['outstanding_short_local_and_NVDR_Volume to Paid Up Shares'] >= -8), 'Short', '0'))
df_action = df4[['Stock','Main Strategy']]
df_action

,Stock,Main Strategy
0,SET:ADVANC,Short
1,SET:AOT,0
2,SET:AWC,0
3,SET:BANPU,0
4,SET:BBL,Short
5,SET:BDMS,0
6,SET:BEM,0
7,SET:BGRIM,Short
8,SET:BH,0
9,SET:BTS,0
